In [22]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy import stats
import time

In [23]:
nifty200=pd.read_csv("ind_nifty200list.csv")

In [24]:
nifty200.columns = ["Company Name","Industry","Symbol","Series","ISIS Code"]
stocksymbollist = nifty200["Symbol"].tolist()

In [29]:
filename=""
for i in range (len(stocksymbollist)):
    filename = stocksymbollist[i]+".ns"
    fileoutname=stocksymbollist[i]+".csv"
    data = yf.download(filename, start="2016-04-01", end="2021-09-23")
    print("stockname:",stocksymbollist[i]," no. of records:",len(data))
    data.to_csv(fileoutname)
    time.sleep(3)

[*********************100%***********************]  1 of 1 completed
stockname: ACC  no. of records: 1350
[*********************100%***********************]  1 of 1 completed
stockname: AUBANK  no. of records: 1036
[*********************100%***********************]  1 of 1 completed
stockname: AARTIIND  no. of records: 1350
[*********************100%***********************]  1 of 1 completed
stockname: ABBOTINDIA  no. of records: 1350
[*********************100%***********************]  1 of 1 completed
stockname: ADANIENT  no. of records: 1350
[*********************100%***********************]  1 of 1 completed
stockname: ADANIGREEN  no. of records: 803
[*********************100%***********************]  1 of 1 completed
stockname: ADANIPORTS  no. of records: 1350
[*********************100%***********************]  1 of 1 completed
stockname: ATGL  no. of records: 709
[*********************100%***********************]  1 of 1 completed
stockname: ADANITRANS  no. of records: 1350
[*****

[*********************100%***********************]  1 of 1 completed
stockname: PFIZER  no. of records: 1350
[*********************100%***********************]  1 of 1 completed
stockname: PIDILITIND  no. of records: 1350
[*********************100%***********************]  1 of 1 completed
stockname: PEL  no. of records: 1350
[*********************100%***********************]  1 of 1 completed
stockname: POLYCAB  no. of records: 600
[*********************100%***********************]  1 of 1 completed
stockname: PFC  no. of records: 1350
[*********************100%***********************]  1 of 1 completed
stockname: POWERGRID  no. of records: 1350
[*********************100%***********************]  1 of 1 completed
stockname: PRESTIGE  no. of records: 1350
[*********************100%***********************]  1 of 1 completed
stockname: PGHH  no. of records: 1350
[*********************100%***********************]  1 of 1 completed
stockname: PNB  no. of records: 1350
[********************

In [25]:
mergedfile=pd.read_csv(stocksymbollist[0]+".csv")
mergedfile["StockSymbol"]=stocksymbollist[0]
i=1
for i in range (len(stocksymbollist)):
    filename = stocksymbollist[i]+".csv"
    f=pd.read_csv(filename)
    f["StockSymbol"]=stocksymbollist[i]
    mergedfile=mergedfile.append(f) 

mergedfile.sort_values(["Date","StockSymbol"], axis = 0, ascending = True, inplace = True, na_position ='last')
mergedfile.to_csv("Finaltrainingdata.csv", index=False)


In [26]:
dateslist = mergedfile["Date"].unique().tolist()

In [27]:
init_amount=1000000
momentum_window=125
minimum_momentum=20
no_of_stocks=20
volatility_window = 20

In [28]:
def Calculate_Momentum(timeseries):
    
    # Make a list of consecutive numbers
    x = np.arange(len(timeseries)) 
    # Get logs
    log_ts = np.log(timeseries) 
    # Calculate regression values
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, log_ts)
    # Annualize percent
    annualized_slope = (np.power((np.exp(slope)), 252) - 1) * 100
    #Adjust for fitness
    momentum = annualized_slope * (r_value ** 2)
    
    return momentum

In [29]:
def Calculate_volatility(timeseries):
    
    tsdf=pd.DataFrame(timeseries)
    volatility = tsdf.pct_change().rolling(volatility_window).std().iloc[-1].values
    volatility = float(volatility)
    if volatility != volatility:
        volatility=0.05
    return volatility

In [30]:
def portfolio_create(mergedlist, date, invest_amount):
   
    #mergedlist filter according to date given
    mergedlistcopy=mergedlist.copy()
    dfdate=mergedlistcopy.loc[mergedlistcopy["Date"] == date].copy()
    stocklist=dfdate["StockSymbol"].tolist()
    
    
    momentumlist=[]
    volatilitylist=[]
    
    #print(len(stocklist), date)
    #dataframe for every stock record in loop
    #delete all records above date
    
    for i in range (len(stocklist)):
        stockcode=stocklist[i]
        stockts=mergedlistcopy.loc[(mergedlistcopy["StockSymbol"] == stockcode) & (mergedlistcopy["Date"] <= date)]
        stocktslist=stockts[-momentum_window:].Close.tolist()
        momentum=Calculate_Momentum(stocktslist)
        momentumlist.append(momentum)
        volatility=Calculate_volatility(stocktslist)
        volatilitylist.append(volatility)

    dfdate["Momentum"]=momentumlist
    dfdate["Volatility"]=volatilitylist
    dfdate["InverseVolatility"]=1/dfdate["Volatility"]
    #sort momentum
    dfdate.sort_values(["Momentum"], axis = 0, ascending = False, inplace = True, na_position ='last')
  
    dfdatefiltered=dfdate[:no_of_stocks].copy()
    
    sumvolatility=dfdatefiltered["InverseVolatility"].sum()
   
    
    dfdatefiltered["Ratio"] = dfdatefiltered["InverseVolatility"]/sumvolatility

    #ivest = init_amount*ratio
    dfdatefiltered["investAmt"]=invest_amount*dfdatefiltered["Ratio"]
    #no.of stcks = invest/closeprice
    dfdatefiltered["numStocks"]= dfdatefiltered["investAmt"]/dfdatefiltered["Close"]
    #convert to int
    #print(dfdatefiltered)
    dfdatefiltered["numStocks"]=dfdatefiltered["numStocks"].astype(int)
    
    #select momentum_window no. of records from end (lastest n records)
    dfdatefiltered["investvalue"]=dfdatefiltered["numStocks"]*dfdatefiltered["Close"]
    dfdatefiltered["Cashremaining"]=dfdatefiltered["investAmt"]-dfdatefiltered["investvalue"]
    dfdatefiltered.drop(["Open", "High", "Low", "Adj Close", "Volume"], axis = 1, inplace = True)
    portfoliovalue=dfdatefiltered["investvalue"].sum()+dfdatefiltered["Cashremaining"].sum()
    dfdatefiltered.to_csv("temp.csv", index = False)
    #print(portfoliovalue)
    
    return dfdatefiltered, portfoliovalue

In [37]:
def evaluation(dfportfolio, mergedlist, date):
    
    mergedlistcopy=mergedlist.copy()
    dfdate=mergedlistcopy.loc[mergedlistcopy["Date"] == date].copy()
    for i in range(len(dfportfolio)):
        stocksymbol=dfportfolio.StockSymbol.values[i]
        count=dfdate.loc[dfdate["StockSymbol"] == stocksymbol].Close.count()
        
        if count <= 0:
            continue
        
        newprice=dfdate.loc[dfdate["StockSymbol"] == stocksymbol].Close.values[0]
        
        #print(stocksymbol, date)
        #dfdate.to_csv("tempdfdate.csv")
        if (newprice == newprice):
            newprice=float(newprice)
            dfportfolio.Close.values[i]=newprice
        
    
    dfportfolio["investvalue"]=dfportfolio["numStocks"]*dfportfolio["Close"]
    portfoliovalue=dfportfolio["investvalue"].sum()+dfportfolio["Cashremaining"].sum()
    
    return dfportfolio, portfoliovalue

In [38]:
def analyze(portfoliohist):
    perf=pd.DataFrame(portfoliohist, columns=['Date', "portfolio_value"])
    perf['max'] = perf.portfolio_value.cummax()
    perf['drawdown'] = (perf.portfolio_value / perf['max']) - 1
    maxdd = perf['drawdown'].min()
    
    ann_ret = (np.power((perf.portfolio_value.iloc[-1] / perf.portfolio_value.iloc[0]),(252 / len(perf)))) - 1
    
    print("Annualized Return: {:.2%} Max Drawdown: {:.2%}".format(ann_ret, maxdd))

       

In [48]:
#main
startdate = "2017-04-01"
enddate = "2021-08-31"
#monthly if statement to check 
    #daily loop all stocks on that day run from startdate to enddate
portfoliohist=[]
daterange=[]

for date in dateslist:
    if (date <=enddate) and (date>=startdate):
        daterange.append(date)

for i in range (len(daterange)):
    print(daterange[i])
    if (i==0):
        dfportfolio, portfoliovalue=portfolio_create(mergedfile, daterange[i], init_amount)
        portrecord=[daterange[i] , portfoliovalue]
      
    else:
        dfportfolio, portfoliovalue=evaluation(dfportfolio, mergedfile, daterange[i]) 
        portrecord=[daterange[i] , portfoliovalue]
        if (i%21==0):
            dfportfolio, portfoliovalue=portfolio_create(mergedfile, daterange[i], portfoliovalue) #rebalancing
    portfoliohist.append(portrecord)
print("Portfolio Value", portfoliovalue)
analyze(portfoliohist)

        

2017-04-03
2017-04-05
2017-04-06
2017-04-07
2017-04-10
2017-04-11
2017-04-12
2017-04-13
2017-04-17
2017-04-18
2017-04-19
2017-04-20
2017-04-21
2017-04-24
2017-04-25
2017-04-26
2017-04-27
2017-04-28
2017-05-02
2017-05-03
2017-05-04
2017-05-05
2017-05-08
2017-05-09
2017-05-10
2017-05-11
2017-05-12
2017-05-15
2017-05-16
2017-05-17
2017-05-18
2017-05-19
2017-05-22
2017-05-23
2017-05-24
2017-05-25
2017-05-26
2017-05-29
2017-05-30
2017-05-31
2017-06-01
2017-06-02
2017-06-05
2017-06-06
2017-06-07
2017-06-08
2017-06-09
2017-06-12
2017-06-13
2017-06-14
2017-06-15
2017-06-16
2017-06-19
2017-06-20
2017-06-21
2017-06-22
2017-06-23
2017-06-27
2017-06-28
2017-06-29
2017-06-30
2017-07-03
2017-07-04
2017-07-05
2017-07-06
2017-07-07
2017-07-10
2017-07-11
2017-07-12
2017-07-13
2017-07-14
2017-07-17
2017-07-18
2017-07-19
2017-07-20
2017-07-21
2017-07-24
2017-07-25
2017-07-26
2017-07-27
2017-07-28
2017-07-31
2017-08-01
2017-08-02
2017-08-03
2017-08-04
2017-08-07
2017-08-08
2017-08-09
2017-08-10
2017-08-11

2020-04-16
2020-04-17
2020-04-20
2020-04-21
2020-04-22
2020-04-23
2020-04-24
2020-04-27
2020-04-28
2020-04-29
2020-04-30
2020-05-04
2020-05-05
2020-05-06
2020-05-07
2020-05-08
2020-05-11
2020-05-12
2020-05-13
2020-05-14
2020-05-15
2020-05-18
2020-05-19
2020-05-20
2020-05-21
2020-05-22
2020-05-26
2020-05-27
2020-05-28
2020-05-29
2020-06-01
2020-06-02
2020-06-03
2020-06-04
2020-06-05
2020-06-08
2020-06-09
2020-06-10
2020-06-11
2020-06-12
2020-06-15
2020-06-16
2020-06-17
2020-06-18
2020-06-19
2020-06-22
2020-06-23
2020-06-24
2020-06-25
2020-06-26
2020-06-29
2020-06-30
2020-07-01
2020-07-02
2020-07-03
2020-07-06
2020-07-07
2020-07-08
2020-07-09
2020-07-10
2020-07-13
2020-07-14
2020-07-15
2020-07-16
2020-07-17
2020-07-20
2020-07-21
2020-07-22
2020-07-23
2020-07-24
2020-07-27
2020-07-28
2020-07-29
2020-07-30
2020-07-31
2020-08-03
2020-08-04
2020-08-05
2020-08-06
2020-08-07
2020-08-10
2020-08-11
2020-08-12
2020-08-13
2020-08-14
2020-08-17
2020-08-18
2020-08-19
2020-08-20
2020-08-21
2020-08-24